In [ ]:
import torch

In [ ]:
device = torch.device("cuda")

In [ ]:
import numpy as np

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.2.1+cu121.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.2.1+cu121.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/10.9 MB ? eta -:--:--Collecting torch-scatter
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 11.8 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
Looking in links: https://data.pyg.org/whl/torch-2.2.1+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 34.0 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-39ufzopu
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-39ufzopu
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-39ufzopu
  Run

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import defaultdict
%matplotlib inline
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


**TRAIN**

with PyG (PyTorch Geometric) library
# autocrine+ GCNG that uses both exocrine and autocrine gene interactions
#For diagonal GCNG, just feed it with a zero matrix instead of adjacent matrix.

In [ ]:
import os
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn import metrics
from numpy import interp

In [ ]:
# Load data
import numpy as np
from scipy import sparse as sp
import pickle
threshold = 140
with open('whole_FOV_distance_I_N_norm_crs_140_norm_lap', 'rb') as fp:
    adj = pickle.load( fp)
#with open('whole_FOV_distance_I_N_crs_140', 'rb') as fp:
    #adj = pickle.load( fp)

In [ ]:
def degree_power(adj, pow):
    """
    Computes \(D^{p}\) from the given adjacency matrix. Useful for computing
    normalised Laplacians.
    :param adj: rank 2 array or sparse matrix
    :param pow: exponent to which elevate the degree matrix
    :return: the exponentiated degree matrix in sparse DIA format
    """
    degrees = np.power(np.array(adj.sum(1)), pow).flatten()
    degrees[np.isinf(degrees)] = 0.
    if sp.issparse(adj):
        D = sp.diags(degrees)
    else:
        D = np.diag(degrees)
    return D


def self_connection_normalized_adjacency(adj, symmetric=True):
    """
    Normalizes the given adjacency matrix using the degree matrix as either
    \(D~^{-1}A~\) or \(D~^{-1/2}A~D~^{-1/2}\) (symmetric normalization).where A~ = A+I
    :param adj: rank 2 array or sparse matrix;
    :param symmetric: boolean, compute symmetric normalization;
    :return: the normalized adjacency matrix.
    """
    if sp.issparse(adj):
        I = sp.eye(adj.shape[-1], dtype=adj.dtype)
    else:
        I = np.eye(adj.shape[-1], dtype=adj.dtype)
    A1 = adj + I
    if symmetric:
        normalized_D = degree_power(A1, -0.5)
        output = normalized_D.dot(A1).dot(normalized_D)
    else:
        normalized_D = degree_power(A1, -1.)
        output = normalized_D.dot(A1)
    return output

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)



MATRIX

In [ ]:
from scipy.sparse import  csr_matrix, coo_matrix
from torch_geometric.utils import to_edge_index, get_laplacian

/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


**FOR EXOCRINE INTERACTION**

In [ ]:
fltr_tensor = sparse_mx_to_torch_sparse_tensor(adj)
fltr_tensor = fltr_tensor.to_sparse_csr() # transform to csr format
edge_index, edge_weight = to_edge_index(fltr_tensor) #edge index  & edge weight
print(edge_weight)

edge_weight = edge_weight.abs()
print(edge_index.shape)
print(edge_weight)


tensor([ 1.0000, -1.0000,  1.0000,  ..., -0.4711, -0.2721,  1.0000])
torch.Size([2, 5396])
tensor([1.0000, 1.0000, 1.0000,  ..., 0.4711, 0.2721, 1.0000])


**FOR AUTOCRINE INTERACTION**

In [ ]:
s = (913,913)
adj_z = np.zeros(s)
fltr = self_connection_normalized_adjacency(adj_z)
f = torch.from_numpy(fltr).to_sparse()
f = f.to_sparse_csr()
edge_index, edge_weight = to_edge_index(f) #edge index (2,913) & edge weight (913)
edge_weight = torch.ones(913)
print(edge_weight)



tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

<ipython-input-11-ee4095c0635f>:5: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  f = f.to_sparse_csr()


**FOR AUTOCRINE AND EXOCRINE INTERACTION**

In [ ]:
# in case of autocrine+
fltr = self_connection_normalized_adjacency(adj)

fltr_tensor = sparse_mx_to_torch_sparse_tensor(fltr) # adj to tensor (coo format)

fltr_tensor = fltr_tensor.to_sparse_csr() # transform to csr format
edge_index, edge_weight = to_edge_index(fltr_tensor) #edge index (2,5403) & edge weight (5403)


<ipython-input-9-2df613f4e512>:46: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:641.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [ ]:
from torch_geometric.nn import GCNConv, GraphConv, GATConv,GATv2Conv, global_mean_pool, global_max_pool
import torch.nn.functional as F

In [ ]:
!pip install torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 7.8 MB/s eta 0:00:00


In [ ]:
from torcheval.metrics import BinaryAccuracy, BinaryAUROC

In [ ]:
# Parameters
l2_reg = 5e-7  # Regularization rate for l2
learning_rate = 1*1e-6  # Learning rate for SGD
batch_size = 16  # Batch size
epochs = 10 # Number of training epochs
es_patience = 5  # Patience fot early stopping
criterion = torch.nn.BCELoss()
pool_batch = torch.zeros(913, dtype=int)


In [ ]:
# Model definition 1
# Epochs: 30
# batch size: 32
class GCN(torch.nn.Module):
      def __init__(self, matrix, weight):
        super().__init__()
        torch.manual_seed(1234567)
        self.adj = matrix
        self.weight = weight
        self.conv1 = GCNConv(in_channels=2, out_channels=32, add_self_loops=False, bias=True)
        self.conv2 = GCNConv(in_channels= 32, out_channels=32, add_self_loops=False, bias=True)
        self.linear1 = torch.nn.Linear(29216,512) #29216
        self.linear2 = torch.nn.Linear(512,1)
        self.activation = torch.nn.ELU()



      def forward(self, x):
        #print("X:", x.shape, "Adj:", adj)
        x = self.conv1(x, self.adj, edge_weight = self.weight)
        x = self.activation(x)
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, self.adj, edge_weight =self.weight)
        x = self.activation(x)
        x = x.flatten(start_dim=1)
        x = self.linear1(x)
        x = x.relu()
        x = self.linear2(x)
        x = x.sigmoid()
        return x



In [ ]:
# Model definition 2
# Epochs: 30
# batch size: 32
class GCN(torch.nn.Module):
      def __init__(self, matrix, weight):
        super().__init__()
        torch.manual_seed(1234567)
        self.adj = matrix
        self.weight = weight
        self.conv1 = GCNConv(in_channels=2, out_channels=32, add_self_loops=False, bias=True)
        self.conv2 = GCNConv(in_channels= 32, out_channels=32, add_self_loops=False, bias=True)
        #self.linear1 = torch.nn.Linear(29216,512) #29216
        self.linear1 = torch.nn.Linear(32,16)
        self.linear3 = torch.nn.Linear(32,32)
        self.linear2 = torch.nn.Linear(16,1)
        self.activation = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(p=0.5)



      def forward(self, x):
        #print("X:", x.shape, "Adj:", adj)
        x = self.conv1(x, self.adj, edge_weight = self.weight)
        x = self.activation(x)
        #x = self.dropout(x)
        x = self.conv2(x, self.adj, edge_weight = self.weight)
        x = self.activation(x)
        #x = self.dropout(x)
        x = self.conv2(x, self.adj, edge_weight =self.weight)
        #x = global_mean_pool(x, pool_batch)
        x = self.activation(x)
        x = x.flatten(start_dim=1)
        #x = self.linear3(x)
        #x = x.relu()
        #x = self.dropout(x)
        x = self.linear1(x)
        x = x.relu()
        #x = self.dropout(x)
        x = self.linear2(x)
        x = x.sigmoid()
        return x

In [ ]:
# Model definition 3
class GCN(torch.nn.Module):
    def __init__(self):

        super().__init__()
        torch.manual_seed(12345678978)
        self.conv1 = GATConv(in_channels=2, out_channels=3, heads=5, add_self_loops=False ,dropout=0)
        self.conv2 = GATConv(in_channels=15,out_channels=3, add_self_loops=False, concat=False, heads=1, dropout=0)
        self.activation = torch.nn.ELU()
        self.dropout = torch.nn.Dropout(p=0.6)
        self.linear1 = torch.nn.Linear(2739,512) #2739
        self.linear2 = torch.nn.Linear(512,1)

    def forward(self, x, edge_index):
        #x, edge_index, z = data[0], data[1], data[2]
        x = self.conv1(x, edge_index)

        x = self.activation(x)
        #x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.activation(x)

        x = x.flatten()


        x = self.linear1(x)
        x = x.relu()
        x = self.linear2(x)

        x = x.sigmoid()

        return x

In [ ]:
# Model definition TEST
# Epochs: 10
# batch size: 16
class GCN(torch.nn.Module):
      def __init__(self, matrix, weight):
        super().__init__()
        torch.manual_seed(1234567)
        self.adj = matrix
        self.weight = weight
        self.conv1 = GraphConv(in_channels=2, out_channels=8, bias=True)
        self.conv2 = GraphConv(in_channels= 8, out_channels=8, bias=True)
        self.linear1 = torch.nn.Linear(7304,512) #29216
        self.linear2 = torch.nn.Linear(512,1)
        self.activation = torch.nn.ELU()



      def forward(self, x):
        #print("X:", x.shape, "Adj:", adj)
        x = self.conv1(x, self.adj, edge_weight = self.weight)
        x = self.activation(x)
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, self.adj, edge_weight =self.weight)
        x = self.activation(x)
        x = x.flatten(start_dim=1)
        x = self.linear1(x)
        x = x.relu()
        x = self.linear2(x)
        x = x.sigmoid()
        return x

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data


In [ ]:
import torch.utils.data as data_utils

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc, precision_recall_curve

In [ ]:
import copy

In [ ]:
def flatten_concatenation(matrix):
     flat_list = []
     for row in matrix:
         flat_list += row
     return flat_list

def ligand_name(pairs):
  pairs = list(pairs)
  o =[]
  for item in pairs:
    if "\\" in r"%r" % item:
      break
    else:
      o.append(item)
  fin = "".join(o)
  return fin

In [ ]:
def train(train_loader):
      out_list = []
      loss_list = []
      model.train()
      for data, label in train_loader:  # Iterate in batches over the training dataset.
        optimizer.zero_grad()  # Clear gradients.
        out = model(data) # Perform a single forward pass.
        out_list.append(out)
        loss = criterion(out, label) # Compute the loss.
        loss_list.append(loss.item())
        loss.backward() # Derive gradients.
        optimizer.step() # Update parameters based on gradients.
      return out_list, loss_list

def train_GAT(trainloader, adj):
      out_list = []
      loss_list = []
      model.train()
      for data, label in trainloader:  # Iterate in batches over the training dataset.
        # for GAT because static graph not supported
        #print(edge_index)
        adj_result = [adj] * len(data)

        te = [Data(x_train=x, edge_index=a) for x,a in zip(data, adj_result)]

        batch = Batch.from_data_list(te)
        #batch = batch.to(device)

        #train_loader = data_utils.DataLoader(t, batch_size=batch_size, shuffle=False)
        optimizer.zero_grad()  # Clear gradients.
        out = model(batch.x_train, batch.edge_index, len(data)) # Perform a single forward pass.
        out_list.append(out)

        loss = criterion(out, label.flatten()) # Compute the loss.

        loss_list.append(loss.item())
        loss.backward() # Derive gradients.
        optimizer.step() # Update parameters based on gradients.
      return out_list, loss_list



def test(test_loader):
     model.eval()

     out_list = []
     for data in test_loader:  # Iterate in batches over the test dataset.
         out = model(data)
         out_list.append(out)

     return out_list

def test_GAT(test_loader, adj):
     model.eval()

     out_list = []
     for data in test_loader:  # Iterate in batches over the test dataset.
         adj_result = [adj] * len(data)

         te = [Data(x_train=x, edge_index=a) for x,a in zip(data, adj_result)]

         batch = Batch.from_data_list(te)
         out = model(batch.x_train, batch.edge_index, len(data))
         out_list.append(out)

     return out_list


In [ ]:
from torch_geometric.data import Data, Batch, DataLoader

In [ ]:
a = np.array([[1],[2],[3]])
b = np.array([[61],[72],[83]])
f = np.append(a,b,axis=0)
print(f)

[[ 1]
 [ 2]
 [ 3]
 [61]
 [72]
 [83]]


In [ ]:
total_accuracy_train = []
total_accuracy_train_final = []
total_accuracy_test_final = []
tn_total = []
fp_total = []
fn_total = []
tp_total = []
tprs = []
aucs = []
w = 1
mean_fpr = np.linspace(0,1,100)
for test_indel in range(1,11): ################## ten fold cross validation
    print("FOLD", test_indel)
    print("----------------------------------------------------------")
    X_data_train = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_train_X_data_array.npy')
    Y_data_train = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_train_Y_data_array.npy')
    gene_pair_index_train = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_train_gene_pair_list_array.npy')
    count_setx_train = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_train_gene_pair_index_array.npy')
    X_data_test = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_test_X_data_array.npy')
    Y_data_test = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_test_Y_data_array.npy')
    gene_pair_index_test = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_test_gene_pair_list_array.npy')
    count_set = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_test_gene_pair_index_array.npy')
    trainX_index = [i for i in range(Y_data_train.shape[0])]
    validation_index = trainX_index[:int(np.ceil(0.2*len(trainX_index)))]
    train_index = trainX_index[int(np.ceil(0.2*len(trainX_index))):]
    X_train, y_train = X_data_train[train_index],Y_data_train[train_index][:,np.newaxis]
    X_val, y_val= X_data_train[validation_index],Y_data_train[validation_index][:,np.newaxis]
    X_test, y_test= X_data_test,Y_data_test[:,np.newaxis]

    # X_train, y_train, X_val, y_val, X_test, y_test, adj = mnist.load_data()
    # X_train, X_val, X_test = X_train[..., None], X_val[..., None], X_test[..., None]
    N = X_train.shape[-2]  # Number of nodes in the graphs
    F = X_train.shape[-1]  # Node features dimensionality
    n_out = y_train.shape[-1]  # Dimension of the target

    ############################################
    #for further training without validation

    X_train = np.append(X_train,X_val,axis=0)

    y_train = np.append(y_train,y_val,axis=0)
    ############################################


    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)

    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)

    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)


    dataset = data_utils.TensorDataset(X_train, y_train)
    val_dataset = data_utils.TensorDataset(X_val, y_val)


    train_loader = data_utils.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    test_loader = data_utils.DataLoader(X_test, batch_size=batch_size, shuffle=False)
    validation_loader = data_utils.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)




    model = GCN(edge_index, edge_weight)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg)

    all_losses = []

    #Initialize Variables for EarlyStopping
    best_loss = float('inf')
    best_model_weights = None
    patience = 5
    t = open('training/'+str(test_indel)+'results.txt', 'w')


    for epoch in range(epochs):
      print("Starting Epoch", epoch)
      accuracy_train_list = []
      output, loss = train(train_loader)

      total_loss = sum(loss) / len(loss)


      # Validation
      #model.eval()  # Set model to evaluation mode
      #with torch.no_grad():  # Disable gradient calculation for validation
          #val_loss_list = []
          #for val_data, val_label in validation_loader:
            #val_outputs = model(val_data)
            #val_loss = criterion(val_outputs, val_label)
            #val_loss_list.append(val_loss)

      #val_loss = sum(val_loss_list) / len(val_loss_list)
      # Early stopping
      #if val_loss < best_loss:
          #best_loss = val_loss
          #best_model_weights = copy.deepcopy(model.state_dict())  # Deep copy here
          #patience = patience  # Reset patience counter
      #else:
          #patience -= 1
          #if patience == 0:
              #break


      best_model_weights = copy.deepcopy(model.state_dict())  # Deep copy here





      # training accuracy, threshold 0.5
      final_out = []
      for out in output:
        output_train_acc = out
        output_train_acc[output_train_acc >= 0.5] = 1
        output_train_acc[output_train_acc < 0.5] = 0
        #output_train_acc_int = int(output_train_acc)   # float to int
        output_train_acc_int = output_train_acc.tolist()
        output_train_acc_int = flatten_concatenation(output_train_acc_int)
        output_train_acc_int = [ int(x) for x in output_train_acc_int ]
        final_out.append(output_train_acc_int)

      final_output = flatten_concatenation(final_out)


      all_losses.append(total_loss)

      accuracy_train = accuracy_score(y_train.flatten(), final_output)
      total_accuracy_train.append(accuracy_train)
      print(f'Epoch: {epoch:03d}, Loss: {total_loss:.4f}, Accuracy: {accuracy_train:.4f}')
      val_loss = 0

      t.write("FOLD" + '\t' + str(test_indel) + '\t' + "EPOCH" + '\t' + str(epoch) +"LOSS" + "\t"+str(total_loss) + "\t"+ "VALIDATION LOSS" + "\t"+str(val_loss) +'\t' + "TRAINING ACCURACY" + '\t'+ str(accuracy_train) + '\n')



    t.close()

    overall_loss = sum(all_losses) / epochs
    print(f'OVERALL LOSS EPOCH: {overall_loss:.4f}')
    sum_train_acc = sum(total_accuracy_train) / epochs
    total_accuracy_train = []
    total_accuracy_train_final.append(sum_train_acc)

    # Load the best model weights
    model.load_state_dict(best_model_weights)

    #############################################
    #              TEST                         #
    #############################################

    print("FOLD", test_indel, "TESTING")

    test_output = test(test_loader)
    final_out = []
    for out in test_output:
      output_test_acc = out
      output_test_acc[output_test_acc >= 0.5] = 1
      output_test_acc[output_test_acc < 0.5] = 0
      #output_train_acc_int = int(output_train_acc)   # float to int
      output_test_acc_int = output_test_acc.tolist()
      output_test_acc_int = flatten_concatenation(output_test_acc_int)
      output_test_acc_int = [ int(x) for x in output_test_acc_int ]
      final_out.append(output_test_acc_int)

    final_output = flatten_concatenation(final_out)

    #Accuracy
    accuracy_test = accuracy_score(y_test.flatten(), final_output)
    total_accuracy_test_final.append(accuracy_test)

    # Confusion Matrix
    cm = confusion_matrix(y_test.flatten(), final_output)
    tn, fp, fn, tp = cm.ravel()
    tn_total.append(tn)
    fp_total.append(fp)
    fn_total.append(fn)
    tp_total.append(tp)


    y_testy = y_test.flatten()
    y_predicty = final_output
    s = open('result/'+str(test_indel)+'divided_interaction.txt', 'w')
    for jj in range(len(count_set) - 1):  # len(count_set)-1):
        if count_set[jj] < count_set[jj + 1]:
            y_test_1 = y_testy[count_set[jj]:count_set[jj + 1]]
            y_predict = y_predicty[count_set[jj]:count_set[jj + 1]]
            # Score trained model.
            cm_1 = confusion_matrix(y_test_1, y_predict)
            tn_1, fp_1, fn_1, tp_1 = cm_1.ravel()
            fpr, tpr, thresholds = metrics.roc_curve(y_test_1, y_predict, pos_label=1)
            auc = metrics.auc(fpr, tpr)
            li_name = ligand_name(gene_pair_index_test[count_set[jj]])
            s.write(li_name + '\n')
            s.write(str(jj) + '\t' + str(count_set[jj]) + '\t' + str(count_set[jj + 1]) + '\t' + str(auc) + '\n')
            s.write(str(jj) + '\t' + str(count_set[jj]) + '\t' + str(count_set[jj + 1]) + '\t' + "TN" + str(tn_1) + "FP" + str(fp_1) + "FN" + str(fn_1) + "TP"+ str(tp_1) + '\n')

    s.close()

    print("FOLD", test_indel, "Test Accuracy:", accuracy_test)
    print("FOLD", test_indel,"Confusion Matrix:", cm)

    fpr, tpr, t = roc_curve(y_test.flatten(), final_output)
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = metrics.auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (w, roc_auc))
    w = w+1


    precision1, recall1, thresholds1 = precision_recall_curve(y_test.flatten(), final_output)
    auc_score_rp = metrics.auc(recall1, precision1)
    print("FOLD", test_indel,"AREA UNDER ROC CURVE:", roc_auc)
    print("FOLD", test_indel,"AREA UNDER P-R CURVE:", auc_score_rp)
    e = open('ending/'+str(test_indel)+'curves.txt', 'w')
    e.write("FOLD" + "\t"+  str(test_indel)+ "\t"+ "TEST ACCURACY:" + "\t"+ str(accuracy_test) + '\n')
    e.write("FOLD" + "\t"+  str(test_indel)+ "\t"+ "TEST CONFUSION MATRIX:" + "\t"+ str(cm) + '\n')
    e.write("FOLD" + "\t"+  str(test_indel)+ "\t"+ "AREA UNDER ROC CURVE:" + "\t"+ str(roc_auc) + '\n')
    e.write("FOLD" + "\t"+  str(test_indel)+ "\t"+ "AREA UNDER P-R CURVE:" + "\t"+ str(auc_score_rp) + '\n')
    e.close()




print("10 FOLD CROSS VALIDATION RESULT")
print("--------------------------------------------------------------")
avg_accuracy_train = sum(total_accuracy_train_final) / 10
avg_accuracy_test = sum(total_accuracy_test_final) / 10
print("Average Training Accuracy", avg_accuracy_train)
print("Average Testing Accuracy", avg_accuracy_test)
print("True Negative:", sum(tn_total), "False Positive:", sum(fp_total), "False Negative:", sum(fn_total), "True Positive:", sum(tp_total))

plt.figure(figsize=(15,15))
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = metrics.auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='blue',
         label=r'Mean ROC (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC')
plt.legend(loc="lower right")
plt.text(0.32,0.7,'More accurate area',fontsize = 12)
plt.text(0.63,0.4,'Less accurate area',fontsize = 12)
plt.show()

FOLD 1
----------------------------------------------------------
Starting Epoch 0
Epoch: 000, Loss: 0.6945, Accuracy: 0.4912
Starting Epoch 1
Epoch: 001, Loss: 0.6924, Accuracy: 0.5243
Starting Epoch 2
Epoch: 002, Loss: 0.6907, Accuracy: 0.5427
Starting Epoch 3
Epoch: 003, Loss: 0.6891, Accuracy: 0.5625
Starting Epoch 4
Epoch: 004, Loss: 0.6876, Accuracy: 0.5836
Starting Epoch 5
Epoch: 005, Loss: 0.6861, Accuracy: 0.6034
Starting Epoch 6
Epoch: 006, Loss: 0.6847, Accuracy: 0.6175
Starting Epoch 7
Epoch: 007, Loss: 0.6833, Accuracy: 0.6288
Starting Epoch 8
Epoch: 008, Loss: 0.6820, Accuracy: 0.6387
Starting Epoch 9
Epoch: 009, Loss: 0.6807, Accuracy: 0.6450
OVERALL LOSS EPOCH: 0.6871
FOLD 1 TESTING
FOLD 1 Test Accuracy: 0.6
FOLD 1 Confusion Matrix: [[2 3]
 [1 4]]
FOLD 1 AREA UNDER ROC CURVE: 0.6000000000000001
FOLD 1 AREA UNDER P-R CURVE: 0.7357142857142857
FOLD 2
----------------------------------------------------------
Starting Epoch 0
Epoch: 000, Loss: 0.6941, Accuracy: 0.4960
Star

**GAT**

# GAT (EINZELN)

In [ ]:
total_accuracy_train = []
total_accuracy_train_final = []
total_accuracy_test_final = []
tn_total = []
fp_total = []
fn_total = []
tp_total = []
tprs = []
aucs = []
w = 1
mean_fpr = np.linspace(0,1,100)
for test_indel in range(1,11): ################## ten fold cross validation
    print("FOLD", test_indel)
    print("----------------------------------------------------------")
    X_data_train = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_train_X_data_array.npy')
    Y_data_train = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_train_Y_data_array.npy')
    gene_pair_index_train = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_train_gene_pair_list_array.npy')
    count_setx_train = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_train_gene_pair_index_array.npy')
    X_data_test = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_test_X_data_array.npy')
    Y_data_test = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_test_Y_data_array.npy')
    gene_pair_index_test = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_test_gene_pair_list_array.npy')
    count_set = np.load('drive/MyDrive/rand_1_10fold_log1p/'+str(test_indel)+'_test_gene_pair_index_array.npy')
    trainX_index = [i for i in range(Y_data_train.shape[0])]
    validation_index = trainX_index[:int(np.ceil(0.2*len(trainX_index)))]
    train_index = trainX_index[int(np.ceil(0.2*len(trainX_index))):]
    X_train, y_train = X_data_train[train_index],Y_data_train[train_index][:,np.newaxis]
    X_val, y_val= X_data_train[validation_index],Y_data_train[validation_index][:,np.newaxis]
    X_test, y_test= X_data_test,Y_data_test[:,np.newaxis]

    # X_train, y_train, X_val, y_val, X_test, y_test, adj = mnist.load_data()
    # X_train, X_val, X_test = X_train[..., None], X_val[..., None], X_test[..., None]
    N = X_train.shape[-2]  # Number of nodes in the graphs
    F = X_train.shape[-1]  # Node features dimensionality
    n_out = y_train.shape[-1]  # Dimension of the target

    ############################################
    #for further training without validation

    X_train = np.append(X_train,X_val,axis=0)

    y_train = np.append(y_train,y_val,axis=0)
    ############################################


    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)

    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)

    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    #dataset = data_utils.TensorDataset(X_train, y_train)

    #train_loader = data_utils.DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # for GAT because static graph not supported
    #print(edge_index)
    #adj_result = [edge_index] *1417
    #stacked_tensor = torch.stack(adj_result, dim=0)


    #te = [Data(x_train=x, edge_index=a, y_train = y) for x,a,y in zip(X_train, adj_result, y_train)]
    #print(te)
    #batch = Batch.from_data_list(te)
    #batch = batch.to(device)

    #train_loader = data_utils.DataLoader(t, batch_size=batch_size, shuffle=False)


    #test_loader = data_utils.DataLoader(X_test, batch_size=batch_size, shuffle=False)

    model = GCN().to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2_reg)

    all_losses = []

    #Initialize Variables for EarlyStopping
    best_loss = float('inf')
    best_model_weights = None
    patience = 5
    t = open('training/'+str(test_indel)+'results.txt', 'w')

    epochs = 8
    for epoch in range(8):
      print("Starting Epoch", epoch)
      accuracy_train_list = []
      out_list = []
      loss_list = []
      model.train()
      for i, data in enumerate(X_train):  # Iterate over the training dataset.
        # for GAT because static graph not supported
        optimizer.zero_grad()  # Clear gradients.
        out = model(data.to(device), edge_index.to(device)) # Perform a single forward pass.
        out_list.append(out)

        loss = criterion(out, y_train[i].to(device)) # Compute the loss.

        loss_list.append(loss.item())
        loss.backward() # Derive gradients.
        optimizer.step() # Update parameters based on gradients.

      total_loss = sum(loss_list) / len(loss_list)
      output = out_list


      # training accuracy, threshold 0.5
      final_out = []
      for out in output:
        output_train_acc = out
        output_train_acc[output_train_acc >= 0.5] = 1
        output_train_acc[output_train_acc < 0.5] = 0
        output_train_acc_int = output_train_acc.tolist()
        output_train_acc_int = [int(x) for x in output_train_acc_int]
        #output_train_acc_int = flatten_concatenation(output_train_acc_int)
        #print(output_train_acc_int)
        #output_train_acc_int = [ int(x) for x in output_train_acc_int ]
        final_out.append(output_train_acc_int)

      final_output = flatten_concatenation(final_out)


      all_losses.append(total_loss)

      accuracy_train = accuracy_score(y_train.flatten(), final_output)
      total_accuracy_train.append(accuracy_train)
      print(f'Epoch: {epoch:03d}, Loss: {total_loss:.4f}, Accuracy: {accuracy_train:.4f}')

      t.write("FOLD" + '\t' + str(test_indel) + '\t' + "EPOCH" + '\t' + str(epoch) +"LOSS" + "\t"+str(total_loss) + "\t"+ '\t' + "TRAINING ACCURACY" + '\t'+ str(accuracy_train) + '\n')



    t.close()

    overall_loss = sum(all_losses) / epochs
    print(f'OVERALL LOSS EPOCH: {overall_loss:.4f}')
    sum_train_acc = sum(total_accuracy_train) / epochs
    total_accuracy_train = []
    total_accuracy_train_final.append(sum_train_acc)

    # Load the best model weights
    #model.load_state_dict(best_model_weights)

    #############################################
    #              TEST                         #
    #############################################

    print("FOLD", test_indel, "TESTING")

    model.eval()

    out_list = []
    for data in X_test:  # Iterate  over the test dataset.
        out = model(data.to(device), edge_index.to(device))
        out_list.append(out)

    test_output = out_list
    final_out = []
    for out in test_output:
      output_test_acc = out
      output_test_acc[output_test_acc >= 0.5] = 1
      output_test_acc[output_test_acc < 0.5] = 0
      #output_train_acc_int = int(output_train_acc)   # float to int
      output_test_acc_int = output_test_acc.tolist()
      #output_test_acc_int = flatten_concatenation(output_test_acc_int)
      output_test_acc_int = [ int(x) for x in output_test_acc_int ]
      final_out.append(output_test_acc_int)

    final_output = flatten_concatenation(final_out)
    #final_output = final_output.to(torch.device("cpu"))


    #Accuracy
    accuracy_test = accuracy_score(y_test.flatten(), final_output)
    total_accuracy_test_final.append(accuracy_test)

    # Confusion Matrix
    cm = confusion_matrix(y_test.flatten(), final_output)
    tn, fp, fn, tp = cm.ravel()
    tn_total.append(tn)
    fp_total.append(fp)
    fn_total.append(fn)
    tp_total.append(tp)


    y_testy = y_test.flatten()
    y_predicty = final_output
    s = open('result/'+str(test_indel)+'divided_interaction.txt', 'w')
    for jj in range(len(count_set) - 1):  # len(count_set)-1):
        if count_set[jj] < count_set[jj + 1]:
            y_test_1 = y_testy[count_set[jj]:count_set[jj + 1]]
            y_predict = y_predicty[count_set[jj]:count_set[jj + 1]]
            # Score trained model.
            cm_1 = confusion_matrix(y_test_1, y_predict)
            tn_1, fp_1, fn_1, tp_1 = cm_1.ravel()
            fpr, tpr, thresholds = metrics.roc_curve(y_test_1, y_predict, pos_label=1)
            auc = metrics.auc(fpr, tpr)
            li_name = ligand_name(gene_pair_index_test[count_set[jj]])
            s.write(li_name + '\n')
            s.write(str(jj) + '\t' + str(count_set[jj]) + '\t' + str(count_set[jj + 1]) + '\t' + str(auc) + '\n')
            s.write(str(jj) + '\t' + str(count_set[jj]) + '\t' + str(count_set[jj + 1]) + '\t' + "TN" + str(tn_1) + "FP" + str(fp_1) + "FN" + str(fn_1) + "TP"+ str(tp_1) + '\n')

    s.close()

    print("FOLD", test_indel, "Test Accuracy:", accuracy_test)
    print("FOLD", test_indel,"Confusion Matrix:", cm)

    fpr, tpr, t = roc_curve(y_test.flatten(), final_output)
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = metrics.auc(fpr, tpr)
    aucs.append(roc_auc)
    plt.plot(fpr, tpr, lw=2, alpha=0.3, label='ROC fold %d (AUC = %0.2f)' % (w, roc_auc))
    w = w+1


    precision1, recall1, thresholds1 = precision_recall_curve(y_test.flatten(), final_output)
    auc_score_rp = metrics.auc(recall1, precision1)
    print("FOLD", test_indel,"AREA UNDER ROC CURVE:", roc_auc)
    print("FOLD", test_indel,"AREA UNDER P-R CURVE:", auc_score_rp)
    e = open('ending/'+str(test_indel)+'curves.txt', 'w')
    e.write("FOLD" + "\t"+  str(test_indel)+ "\t"+ "TEST ACCURACY:" + "\t"+ str(accuracy_test) + '\n')
    e.write("FOLD" + "\t"+  str(test_indel)+ "\t"+ "TEST CONFUSION MATRIX:" + "\t"+ str(cm) + '\n')
    e.write("FOLD" + "\t"+  str(test_indel)+ "\t"+ "AREA UNDER ROC CURVE:" + "\t"+ str(roc_auc) + '\n')
    e.write("FOLD" + "\t"+  str(test_indel)+ "\t"+ "AREA UNDER P-R CURVE:" + "\t"+ str(auc_score_rp) + '\n')
    e.close()




print("10 FOLD CROSS VALIDATION RESULT")
print("--------------------------------------------------------------")
avg_accuracy_train = sum(total_accuracy_train_final) / 10
avg_accuracy_test = sum(total_accuracy_test_final) / 10
print("Average Training Accuracy", avg_accuracy_train)
print("Average Testing Accuracy", avg_accuracy_test)
print("True Negative:", sum(tn_total), "False Positive:", sum(fp_total), "False Negative:", sum(fn_total), "True Positive:", sum(tp_total))


FOLD 1
----------------------------------------------------------
Starting Epoch 0
Epoch: 000, Loss: 0.6916, Accuracy: 0.5655
Starting Epoch 1
Epoch: 001, Loss: 0.6879, Accuracy: 0.5897
Starting Epoch 2
Epoch: 002, Loss: 0.6849, Accuracy: 0.5903
Starting Epoch 3
Epoch: 003, Loss: 0.6822, Accuracy: 0.6134
Starting Epoch 4
Epoch: 004, Loss: 0.6796, Accuracy: 0.6281
Starting Epoch 5
Epoch: 005, Loss: 0.6771, Accuracy: 0.6462
Starting Epoch 6
Epoch: 006, Loss: 0.6748, Accuracy: 0.6659
Starting Epoch 7
Epoch: 007, Loss: 0.6725, Accuracy: 0.6772
OVERALL LOSS EPOCH: 0.6813
FOLD 1 TESTING
FOLD 1 Test Accuracy: 0.6
FOLD 1 Confusion Matrix: [[3 2]
 [2 3]]
FOLD 1 AREA UNDER ROC CURVE: 0.6
FOLD 1 AREA UNDER P-R CURVE: 0.7
FOLD 2
----------------------------------------------------------
Starting Epoch 0
Epoch: 000, Loss: 0.6918, Accuracy: 0.5453
Starting Epoch 1
Epoch: 001, Loss: 0.6881, Accuracy: 0.6144
Starting Epoch 2
Epoch: 002, Loss: 0.6850, Accuracy: 0.6376
Starting Epoch 3
Epoch: 003, Loss:

**NOTIZEN**